In [12]:
import pdb
import pickle
import string
import time
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
from nltk.corpus import stopwords, twitter_samples
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
from os import getcwd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from collections import defaultdict
from nltk.stem import PorterStemmer

In [13]:
en_embeddings_subset = pickle.load(open("../data/en_embeddings.p", "rb"))

In [15]:
filePath = f"{getcwd()}/data/"
nltk.data.path.append(filePath)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...


In [16]:
# tags needed for lemmatization
tag_dict = defaultdict(lambda : wordnet.NOUN)
tag_dict['J'] = wordnet.ADJ
tag_dict['V'] = wordnet.VERB
tag_dict['R'] = wordnet.ADV

def process_tweet(text):
    
    tweet = text.lower()
    tweet = re.sub(r'[0-9]+', '', tweet)
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    tweet = tweet.translate((None, string.punctuation))
    
  
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    stopwords_english = stopwords.words('english')
    tweet = [word for word in tweet_tokens if word not in stopwords_english]
    stemmer = PorterStemmer()
    tweets_clean = []
    
    lemmatizer=WordNetLemmatizer()
    lemmatize_text = []
    for token, tag in pos_tag(tweet):
        word = lemmatizer.lemmatize(token, tag_dict[tag[0]])
        stem_word = stemmer.stem(word)  # stemming word
        if(len(stem_word)>1):
            tweets_clean.append(stem_word)
    
    return tweets_clean

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/tf/NLP/NLP/NLP with Classification and Vector Spaces/data/'
**********************************************************************


In [ ]:
def get_document_embedding(doc,embedding):
    
    doc_embedding = np.zeros(300)
    clean_doc = process_tweet(doc)
    for word in clean_doc:
        doc_embedding += embedding.get(word,0)
        
    return doc_embedding

In [ ]:
def get_document_vecs(all_docs, en_embeddings):
  
    ind2Doc_dict = {}
    document_vec_l = []
    for i, doc in enumerate(all_docs):
        doc_embedding = get_document_embedding(doc,en_embeddings)
        ind2Doc_dict[i] = doc_embedding

        document_vec_l.append(doc_embedding)
    document_vec_matrix = np.vstack(document_vec_l)

    return document_vec_matrix, ind2Doc_dict